# Deep Learning Playground

We are interested in the following features:

- H1: region-averaged time-series
- H2: region-level connectivities (from H1, optional: triu-k1)
- H3: network-averaged time-series (from H1)
- H4: network connectivity (from H3, optional: triu-k1)
- H5: networks connectivity (from H2, optional: triu-k0)

> Note that we are not going to take the upper triangular part of the connectivity matrix and full matrices are used instead.

In [3]:
%reload_ext autoreload
%autoreload 2

import pytorch_lightning as pl
from pytorch_lightning.callbacks import RichProgressBar

from src.acnets.deep import ACNetsDataModule, MultiHeadModel

In [4]:
datamodule = ACNetsDataModule(atlas='dosenbach2010', kind='partial correlation', batch_size=8)
datamodule.setup()

n_regions = datamodule.train[0][0].shape[1]
n_networks = datamodule.train[0][2].shape[1]

# DEBUG single epoch, one train/test split
model = MultiHeadModel(n_regions, n_networks, n_embeddings=16)
trainer = pl.Trainer(accelerator='auto', max_epochs=10000,
                     accumulate_grad_batches=3,
                     log_every_n_steps=3, callbacks=[RichProgressBar()])
trainer.fit(model, datamodule)
trainer.test(model, datamodule=datamodule)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name     ┃ Type               ┃ Params ┃
┡━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ accuracy │ MulticlassAccuracy │      0 │
│ 1 │ x4_head  │ Sequential         │  1.0 K │
│ 2 │ x5_head  │ Sequential         │  1.3 K │
│ 3 │ cls_head │ Sequential         │    154 │
└───┴──────────┴────────────────────┴────────┘

Trainable params: 2.5 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 2.5 K                                                                                                
Total estimated model params size (MB): 0

Output()

/home/morteza/micromamba/envs/acnets/lib/python3.11/site-packages/pytorch_lightning/trainer/call.py:54: Detected 
KeyboardInterrupt, attempting graceful shutdown...

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test/accuracy       │            0.5            │
│   test/dropout_accuracy   │     0.550000011920929     │
│       test/loss_cls       │    0.6922729015350342     │
└───────────────────────────┴───────────────────────────┘

[{'test/loss_cls': 0.6922729015350342,
  'test/accuracy': 0.5,
  'test/dropout_accuracy': 0.550000011920929}]